In [9]:

"""
@author: yxzhang
This is the part of the numerical study in [Phys. Rev. A 109, 022606]
In paticular we use snap + displacement + SU(2) layers to synthesize a target unitary
"""

import time
from scipy.optimize import minimize, dual_annealing, basinhopping
from circuit import Circuit
import numpy as np
from numpy.linalg import lstsq
from scipy.linalg import orth
import pickle
from scipy import linalg as la
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

def load_from_disc(loc):
    filehandler = open(loc, "rb")
    v = pickle.load(filehandler)
    return v

def save_to_disc(v,loc):
    filehandler = open(loc,"wb")
    pickle.dump(v,filehandler)
    filehandler.close()

qdim = 2 # qubit dimension
bdim = 6 # cavity levels; set to be equal to the bond dimension for now

r_max = 10
isotype = "rdm"
#isotype = "u_ising"
from scipy.stats import unitary_group
O  = np.kron(np.identity(2),unitary_group.rvs(bdim)) #taken to be a random unitary in the cavity
# target function:
def C_U(V):# the log of the infidelity
    return np.log10((1 - abs(np.trace(V.T @ O))/len(O)))

def C_U_nonlog(V):# infidelity
    return abs(1 - abs(np.trace(V.T @ O))/len(O))

def C_U_params(params):
    unitary = c.get_tensor(params)
    unitary = unitary.swapaxes(0,1)
    unitary = unitary.swapaxes(2,3)
    U1 = unitary.reshape((qdim*bdim, qdim*bdim))
    return C_U(U1)

def C_U_params_nonlog(params):
    unitary = c.get_tensor(params)
    unitary = unitary.swapaxes(0,1)
    unitary = unitary.swapaxes(2,3)
    U1 = unitary.reshape((qdim*bdim, qdim*bdim))
    return C_U_nonlog(U1)

def layer_circ(round_num):
    c = Circuit([("qubit", "p", qdim), ("cavity", "b", bdim)])
    c.add_gate("displacement")
    for k in range(round_num):
        # adding layers of gates
        c.add_gate("snap")
        c.add_gate("rotation")
        c.add_gate("cpm", qids = [0,1], n_params=0, fn=lambda p:p)
        #c.add_gate("h", qids = [0])
        c.add_gate("displacement")
    c.assemble()
    return c

#O = Uni_Larger(MPS_to_Uni(bdim,qdim,iso6), cdim*qdim, bdim*qdim)
#initial optimization:
results = []

r_num = 2 #number of layers
randomness = 0.1 # how much randomness to use in initialization; identity otherwise
c = layer_circ(r_num)
rng = np.random.default_rng()
params = rng.uniform(high=2*np.pi, size=c.n_params)
t1 = time.time()
result = minimize(C_U_params, x0=params, method='nelder-mead')
sweet_spot = result.x
#save_to_disc(sweet_spot, "target_" + isotype+ "_b" + str(bdim)+"params_round_"+str(r_num))
t2 = time.time()
#print(sweet_spot)

params_prev = result.x

#run the iterations:
for r_num in range(2,r_max,2):
    c = layer_circ(r_num)
    #load_from_disc("target_" + isotype+ "_b" + str(bdim)+"params_round_"+str(r_num-1))
    rng = np.random.default_rng()
    params = rng.uniform(high=2*np.pi, size=c.n_params) * randomness
    for i in range(len(params_prev)):
        params[i] = params[i] + params_prev[i]
    t1 = time.time()
    result = minimize(C_U_params, x0=params, method='nelder-mead')
    sweet_spot = result.x
    params_prev = result.x
    results.append(result.fun)
    #save_to_disc(sweet_spot, "target_" + isotype+ "_b" + str(bdim)+"params_round_"+str(r_num))
    t2 = time.time()
    print(r_num, result.fun)
print(10**np.array(results))


2 -0.2420210632450816
4 -0.34465469654318337
6 -0.5389179870909955
8 -0.592659623015482
[0.573 0.452 0.289 0.255]
